### Pandas: merging by closest date example

In [119]:
import pandas as pd

In [120]:
left = pd.DataFrame({
    "ID": [1, 2, 3, 4, 5], 
    "date":[
        "06/22/2014",
        "07/02/2014",
        "01/01/2015",
        "01/01/1991",
        "08/02/1999"
    ]})

right = pd.DataFrame({
    "ID": [1, 2, 3, 4, 5, 6, 7], 
    "date":[
        "02/15/2015",
        "06/30/2014",
        "07/02/1999",
        "10/05/1990",
        "06/24/2014",
        "12/12/2222",
        "12/13/2222"
    ], "value": [333, 222, 444, 777, 42, 0, 0] 
})

left['date']  = pd.to_datetime(left.date)
right['date'] = pd.to_datetime(right.date)
left.sort_values('date', inplace=True)
right.sort_values('date', inplace=True)

In [121]:
left, right

(   ID       date
 3   4 1991-01-01
 4   5 1999-08-02
 0   1 2014-06-22
 1   2 2014-07-02
 2   3 2015-01-01,
    ID       date  value
 3   4 1990-10-05    777
 2   3 1999-07-02    444
 4   5 2014-06-24     42
 1   2 2014-06-30    222
 0   1 2015-02-15    333
 5   6 2222-12-12      0
 6   7 2222-12-13      0)

In [122]:
right1 = right.set_index('date').reindex(left.set_index('date').index, method='nearest').reset_index()

In [123]:
pd.merge(left, right1, on='date', suffixes=('_a', '_b'))

,ID_a,date,ID_b,value
0,4,1991-01-01,4,777
1,5,1999-08-02,3,444
2,1,2014-06-22,5,42
3,2,2014-07-02,2,222
4,3,2015-01-01,1,333


### Engine data

In [124]:
egtm = pd.read_csv('./EGTM.csv')

In [125]:
egtm_cols = egtm.columns[:5]
egtm = egtm[egtm_cols]
egtm = egtm[egtm['AIRCRAFT'] == 'VQ-BDQ']
egtm = egtm[egtm['POSITION'] == 1]
egtm = egtm.rename(columns={'RECORDED_DT': 'reportts'})

egtm['reportts'] = pd.to_datetime(egtm['reportts'], format='mixed')
egtm['egtm_reportts'] = egtm['reportts'] 


In [126]:
neo = pd.read_csv(
    './a320_a321_neo_full_acms_parameters.csv', 
    parse_dates=['reportts', 'processedts'], 
)
neo = neo[neo['acnum'] == 'VQ-BDQ']
neo = neo[neo['pos'] == 1]

/tmp/ipykernel_7741/3501257036.py:1: DtypeWarning: Columns (152,154,172,173,176,177,178) have mixed types. Specify dtype option on import or set low_memory=False.
  neo = pd.read_csv(


In [127]:
neo.head()

,acnum,reportname,reportts,file_path,pos,fltdes,dep,arr,pf,wai,...,tr,ttp,vb1,vb2,vorrc,vorv,votm,vsva,w14,processedts
0,VQ-BDQ,A320 ENG GAS PATH ADVISORY REPORT,2018-05-17 09:29:45,ecm/S7_NEO_320/VQ-BDQ/VQ-BDQ18052018135429R/_V...,1,9993,LFBO,UUDD,0.09,0.0,...,NaN,0.0,0.0,0.1,83.9,NaN,27.1,0.099,242.0,2023-09-25 10:17:19.928
2,VQ-BDQ,A320 ENG GAS PATH ADVISORY REPORT,2018-05-17 09:31:48,ecm/S7_NEO_320/VQ-BDQ/VQ-BDQ18052018135429R/_V...,1,9993,LFBO,UUDD,0.54,0.0,...,NaN,0.0,0.0,0.1,79.2,NaN,27.5,0.114,244.0,2023-09-25 10:17:20.128
4,VQ-BDQ,A320 ENGINE CRUISE REPORT,2018-05-17 12:42:09,ecm/S7_NEO_320/VQ-BDQ/VQ-BDQ18052018135429R/_V...,1,9993,LFBO,UUDD,0.48,0.0,...,NaN,NaN,NaN,NaN,39.1,39.1,NaN,0.765,562.0,2023-09-25 10:17:19.614
6,VQ-BDQ,A320 EEC MAINTENANCE REPORT,2018-05-17 13:41:56,ecm/S7_NEO_320/VQ-BDQ/VQ-BDQ18052018135452R/_V...,1,9993,LFBO,UUDD,0.48,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,24.5,0.275,NaN,2023-09-25 10:17:20.409
8,VQ-BDQ,A320 EEC MAINTENANCE REPORT,2018-05-17 13:42:10,ecm/S7_NEO_320/VQ-BDQ/VQ-BDQ18052018135452R/_V...,1,9993,LFBO,UUDD,0.48,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,25.2,0.596,NaN,2023-09-25 10:17:20.544


#### Merging with EGTM table by closest recorded date (using indexes)

In [128]:
neo.sort_values('reportts', inplace=True)
egtm.sort_values('reportts', inplace=True)

egtm = egtm.set_index('reportts')
neo = neo.set_index('reportts')

In [129]:
egtm.index

DatetimeIndex(['2018-01-06 22:12:00', '2018-01-07 08:26:00',
               '2018-01-07 13:56:00', '2018-01-07 19:40:00',
               '2018-01-08 03:01:00', '2018-01-08 08:41:00',
               '2018-01-08 14:11:00', '2018-01-08 19:45:00',
               '2018-01-09 05:11:00', '2018-01-09 10:39:00',
               ...
               '2020-08-01 16:16:00', '2020-09-01 07:59:00',
               '2020-09-01 16:49:00', '2020-10-01 04:01:00',
               '2020-10-01 07:36:00', '2020-11-01 05:46:00',
               '2020-11-01 21:33:00', '2020-12-01 00:02:00',
               '2020-12-01 05:05:00', '2020-12-01 13:11:00'],
              dtype='datetime64[ns]', name='reportts', length=1284, freq=None)

In [130]:
egtm_indexed = egtm.reindex(neo.index, method='nearest').reset_index()

In [131]:
pd.merge(neo, egtm_indexed, on='reportts')

,reportts,acnum,reportname,file_path,pos,fltdes,dep,arr,pf,wai,...,vorv,votm,vsva,w14,processedts,AIRCRAFT,POSITION,ESN,RWCTOMAR,egtm_reportts
0,2018-05-17 09:29:45,VQ-BDQ,A320 ENG GAS PATH ADVISORY REPORT,ecm/S7_NEO_320/VQ-BDQ/VQ-BDQ18052018135429R/_V...,1,9993,LFBO,UUDD,0.09,0.0,...,NaN,27.1,0.099,242.0,2023-09-25 10:17:19.928,VQ-BDQ,1,770664,132.144,2018-05-20 19:36:47
1,2018-05-17 09:31:48,VQ-BDQ,A320 ENG GAS PATH ADVISORY REPORT,ecm/S7_NEO_320/VQ-BDQ/VQ-BDQ18052018135429R/_V...,1,9993,LFBO,UUDD,0.54,0.0,...,NaN,27.5,0.114,244.0,2023-09-25 10:17:20.128,VQ-BDQ,1,770664,132.144,2018-05-20 19:36:47
2,2018-05-17 12:42:09,VQ-BDQ,A320 ENGINE CRUISE REPORT,ecm/S7_NEO_320/VQ-BDQ/VQ-BDQ18052018135429R/_V...,1,9993,LFBO,UUDD,0.48,0.0,...,39.1,NaN,0.765,562.0,2023-09-25 10:17:19.614,VQ-BDQ,1,770664,132.144,2018-05-20 19:36:47
3,2018-05-17 13:41:56,VQ-BDQ,A320 EEC MAINTENANCE REPORT,ecm/S7_NEO_320/VQ-BDQ/VQ-BDQ18052018135452R/_V...,1,9993,LFBO,UUDD,0.48,0.0,...,NaN,24.5,0.275,NaN,2023-09-25 10:17:20.409,VQ-BDQ,1,770664,132.144,2018-05-20 19:36:47
4,2018-05-17 13:42:10,VQ-BDQ,A320 EEC MAINTENANCE REPORT,ecm/S7_NEO_320/VQ-BDQ/VQ-BDQ18052018135452R/_V...,1,9993,LFBO,UUDD,0.48,0.0,...,NaN,25.2,0.596,NaN,2023-09-25 10:17:20.544,VQ-BDQ,1,770664,132.144,2018-05-20 19:36:47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20992,2025-04-21 07:34:13,VQ-BDQ,A320 EEC MAINTENANCE REPORT,adw-data/aircraft/pdm/data/qar/binary/A320NEO/...,1,3034,UIUU,UUDD,0.42,0.0,...,NaN,26.3,0.330,NaN,2023-09-24 02:46:02.330,VQ-BDQ,1,770664,106.968,2020-12-01 13:11:00
20993,2025-04-21 07:38:38,VQ-BDQ,A320 EEC MAINTENANCE REPORT,adw-data/aircraft/pdm/data/qar/binary/A320NEO/...,1,3034,UIUU,UUDD,0.46,0.0,...,NaN,35.9,0.261,NaN,2023-09-24 02:46:02.487,VQ-BDQ,1,770664,106.968,2020-12-01 13:11:00
20994,2025-04-21 08:08:36,VQ-BDQ,A320 EEC MAINTENANCE REPORT,adw-data/aircraft/pdm/data/qar/binary/A320NEO/...,1,3034,UIUU,UUDD,0.54,0.0,...,NaN,27.3,0.216,NaN,2023-09-24 02:46:02.614,VQ-BDQ,1,770664,106.968,2020-12-01 13:11:00
20995,2025-04-21 08:08:49,VQ-BDQ,A320 EEC MAINTENANCE REPORT,adw-data/aircraft/pdm/data/qar/binary/A320NEO/...,1,3034,UIUU,UUDD,0.50,0.0,...,NaN,26.5,0.204,NaN,2023-09-24 02:46:02.840,VQ-BDQ,1,770664,106.968,2020-12-01 13:11:00
